#### Conceptually: What We're Building

After connecting three perspectives:

- Physical (JRC Metadata) → what Finland has (plants, MW).

- Climatic (Zenodo) → what nature could provide (inflow potential).

- Operational (Fingrid) → what’s actually being generated right now.

Then computing efficiency and deviation metrics like:

Are we producing as much as expected given our long-term climatic inflows and installed capacity?

#### Compute Observed Capacity Factor

Computed Observed Capacity Factor using Fingrid ÷ total installed capacity (Gold Layer #1)

In [9]:
USE hydro_energy_lakehouse;

-- Step 1: Compute total installed capacity (MW)
CREATE OR REPLACE TEMP VIEW total_capacity AS
SELECT
    SUM(total_capacity_mw) AS total_capacity_mw
FROM silver_meta_capacity_fi;


-- Step 2: Calculate observed capacity factor and store in Gold table
CREATE OR REPLACE TABLE gold_observed_cf AS
SELECT
    f.hour,
    f.generation_mw,
    (f.generation_mw / t.total_capacity_mw) AS observed_cf
FROM
    silver_fingrid_hourly AS f
CROSS JOIN
    total_capacity AS t
ORDER BY
    f.hour;

StatementMeta(, 300338c3-ab13-4986-8654-ecac1fb33322, 16, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [1]:
-- Gold layer join
CREATE OR REPLACE TABLE gold_hydro_cf AS
SELECT 
    f.hour AS timestamp,
    f.generation_mw,
    (f.generation_mw / m.total_capacity_mw_total) AS observed_cf,
    z.avg_cf AS historical_cf,
    (f.generation_mw / m.total_capacity_mw_total) - z.avg_cf AS deviation
FROM silver_fingrid_hourly f
CROSS JOIN (
    SELECT SUM(total_capacity_mw) AS total_capacity_mw_total 
    FROM silver_meta_capacity_fi
) m
LEFT JOIN silver_zenodo_fi_cf z
    ON HOUR(f.hour) = HOUR(z.timestamp)
    AND MONTH(f.hour) = MONTH(z.timestamp);


StatementMeta(, 300338c3-ab13-4986-8654-ecac1fb33322, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

### Logic:

- We join Fingrid’s recent 10 days with Zenodo’s long-term hourly pattern (month + hour alignment).

- We use total installed capacity from metadata as denominator.

- The result gives a comparable “capacity factor anomaly” for each hour.

In [3]:
SHOW TABLES FROM hydro_energy_lakehouse

StatementMeta(, 300338c3-ab13-4986-8654-ecac1fb33322, 4, Finished, Available, Finished)

<Spark SQL result set with 7 rows and 3 fields>

In [5]:
USE hydro_energy_lakehouse;

SELECT *
FROM gold_hydro_cf
LIMIT 10;

StatementMeta(, 300338c3-ab13-4986-8654-ecac1fb33322, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 10 rows and 5 fields>

## Calculation of Monthly Baselines

As we can see there is much difference between observed and historical. So we need to have some granularity for this time-series data (Gold Layer #2)

In [7]:
-- Use the correct Lakehouse context
USE hydro_energy_lakehouse;

-- Create a Gold table with monthly averages & standard deviations
CREATE OR REPLACE TABLE gold_zenodo_baseline_monthly
AS
SELECT
    YEAR(timestamp)  AS year,
    MONTH(timestamp) AS month,
    AVG(avg_cf)      AS baseline_avg_cf,
    STDDEV(avg_cf)   AS baseline_std_cf
FROM silver_zenodo_fi_cf
GROUP BY
    YEAR(timestamp),
    MONTH(timestamp)
ORDER BY
    YEAR(timestamp),
    MONTH(timestamp);



StatementMeta(, 300338c3-ab13-4986-8654-ecac1fb33322, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [10]:
USE hydro_energy_lakehouse;

-- Create or replace the Gold-level table for historical hourly patterns
CREATE OR REPLACE TABLE gold_zenodo_baseline_hourlypattern AS
SELECT
    MONTH(timestamp) AS month,
    HOUR(timestamp)  AS hour_of_day,
    AVG(avg_cf)      AS historical_avg_cf,
    AVG(avg_ror_cf)  AS historical_ror_cf,
    AVG(avg_sto_cf)  AS historical_sto_cf
FROM
    silver_zenodo_fi_cf
GROUP BY
    MONTH(timestamp),
    HOUR(timestamp)
ORDER BY
    MONTH(timestamp),
    HOUR(timestamp);

StatementMeta(, 300338c3-ab13-4986-8654-ecac1fb33322, 18, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

This is our baseline climatology — a 12 × 24 grid = 288 points (month-hour averages).

In [12]:
USE hydro_energy_lakehouse;

-- Create (or overwrite) the Gold table for observed vs. historical CF deviation
CREATE OR REPLACE TABLE gold_cf_deviation AS
SELECT
    o.hour,
    o.observed_cf,
    b.historical_avg_cf,
    (o.observed_cf - b.historical_avg_cf) AS deviation_cf
FROM
    gold_observed_cf AS o
LEFT JOIN
    gold_zenodo_baseline_hourlypattern AS b
ON
    MONTH(o.hour) = b.month
    AND HOUR(o.hour) = b.hour_of_day
ORDER BY
    o.hour;

StatementMeta(, 300338c3-ab13-4986-8654-ecac1fb33322, 21, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [13]:
SELECT * FROM gold_cf_deviation LIMIT 20;

StatementMeta(, 300338c3-ab13-4986-8654-ecac1fb33322, 22, Finished, Available, Finished)

<Spark SQL result set with 20 rows and 4 fields>

In [14]:
USE hydro_energy_lakehouse;

-- Create or overwrite the monthly aggregated gold-level table for CF deviations
CREATE OR REPLACE TABLE gold_cf_deviation_monthly AS
SELECT
    MONTH(hour) AS month,
    AVG(observed_cf)       AS avg_observed_cf,
    AVG(historical_avg_cf) AS avg_historical_cf,
    AVG(deviation_cf)      AS avg_deviation
FROM
    gold_cf_deviation
GROUP BY
    MONTH(hour)
ORDER BY
    month;

StatementMeta(, 300338c3-ab13-4986-8654-ecac1fb33322, 24, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [15]:
SELECT * FROM gold_cf_deviation_monthly Limit 1000; 

StatementMeta(, 300338c3-ab13-4986-8654-ecac1fb33322, 25, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>